In [136]:
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization,Conv3D,Dropout, MaxPooling3D, Flatten, Dense,Concatenate, Reshape, TimeDistributed, Input, concatenate, LSTM,  Conv2D, MaxPooling2D, RepeatVector
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow import keras
import matplotlib.pyplot as plt

In [12]:
# 서울 PM2.5 초미세먼지 데이터 불러오기
pm_data = pd.read_csv("./daily_seoul_pm25.csv", header = [0, 1], index_col = 0)

In [13]:
pm_data.head()

,PM25
지역,서울 서대문구
2019-01-01,26.6
2019-01-02,21.4
2019-01-03,23.9
2019-01-04,40.1
2019-01-05,46.5


In [14]:
# 인덱스를 datetime 형식으로 변환
pm_data.index = pd.to_datetime(pm_data.index)

In [31]:
# 2021년 01월 01일부터 사용
pm_data = pm_data.loc["2021-01-01":]

In [34]:
pm_data.shape

(1235, 1)

In [39]:
# 국내 기상데이터 불러오기
climate_data = pd.read_csv("C:/Windows/System32/01.Final_project/data/pm25_final.csv", index_col = 0)

In [41]:
# 인덱스를 datetime 형식으로 변환
climate_data.index = pd.to_datetime(climate_data.index)

In [42]:
climate_data.head()

,"('PM25', '강원 강릉시')","('PM25', '광주 북구')","('PM25', '대전 중구')","('PM25', '부산 해운대구')","('PM25', '서울 서대문구')","('강수량', '90')","('강수량', '101')","('강수량', '102')","('강수량', '105')","('강수량', '108')",...,month_11,month_12,hour_0,hour_3,hour_6,hour_9,hour_12,hour_15,hour_18,hour_21
2019-01-01 03:00:00,15.0,20.3,39.0,19.3,33.7,0.0,0.0,0.0,0.0,0.0,...,False,False,False,True,False,False,False,False,False,False
2019-01-01 06:00:00,10.7,22.0,36.0,19.7,32.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,True,False,False,False,False,False
2019-01-01 09:00:00,14.0,21.7,31.0,22.0,29.3,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,True,False,False,False,False
2019-01-01 12:00:00,14.0,18.3,29.0,17.0,22.7,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,True,False,False,False
2019-01-01 15:00:00,9.5,15.3,17.7,15.3,27.3,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,True,False,False


In [43]:
climate_data.shape

(15727, 2225)

In [45]:
# 종속변수들은 제거
climate_data = climate_data.iloc[:, 5:]

In [52]:
# 날짜별로 평균내서 그룹화하기
grouped_climate = climate_data.groupby(climate_data.index.date)
grouped_climate = grouped_climate.mean()

In [53]:
grouped_climate.shape

(1966, 2220)

In [60]:
# 인덱스를 datetime 형식으로 변환
grouped_climate.index = pd.to_datetime(grouped_climate.index)

In [62]:
# 2021년 01월 01일부터 추출
grouped_climate = grouped_climate.loc["2021-01-01":,:]

In [63]:
grouped_climate.shape

(1235, 2220)

In [66]:
# 에어로졸 이미지 불러오기
img_df = pd.read_csv(r"C:\Windows\System32\01.Final_project\jayden\final_aerosol_df.csv", index_col = 0)

In [67]:
# 인덱스를 datetime 형식으로 변환
img_df.index = pd.to_datetime(img_df.index)

In [68]:
# 기상,미세먼지 데이터와 동일하게 2024-05-19 까지만 추출
img_df = img_df.loc[:"2024-05-19 02:45:00",:]

In [85]:
img_df.shape

(3699, 1)

In [1]:
3699 / 3

1233.0

In [76]:
# 에어로졸 이미지 중, 특정한 날짜 2개에 결측치가 있는 것으로 보임. 어느 날짜가 결측인지 확인하기위해 모든 날짜를 생성해서 비교해보기.

# 2021년01월01일 00:45:00 부터 2024년 5월19일 02:25:00 까지 생성
date_range = pd.date_range(start="2021-01-01 00:45:00", end="2024-05-19 02:45:00", freq='D')

# 각 날짜에 대해 세 개의 시간대를 포함하는 리스트를 생성
date_series = pd.Series(
    pd.date_range(start="2021-01-01 00:45:00", end="2024-05-19 02:45:00", freq='D').strftime('%Y-%m-%d')
).apply(lambda date: [f"{date} 00:45:00", f"{date} 01:45:00", f"{date} 02:45:00"])

# explode 메소드를 사용하여 각 날짜별 시간 리스트를 평탄화하여 단일 시리즈로 변환 후 인덱스는 drop
flat_series = date_series.explode().reset_index(drop=True)

# pd.to_datetime 함수를 사용하여 평탄화된 문자열 시리즈를 datetime 형식으로 변환
datetime_series = pd.to_datetime(flat_series)

# 변환된 datetime 시리즈를 데이터프레임으로 생성
datetime_df = pd.DataFrame(datetime_series, columns=["DateTime"])
print(datetime_df)

                DateTime
0    2021-01-01 00:45:00
1    2021-01-01 01:45:00
2    2021-01-01 02:45:00
3    2021-01-02 00:45:00
4    2021-01-02 01:45:00
...                  ...
3700 2024-05-18 01:45:00
3701 2024-05-18 02:45:00
3702 2024-05-19 00:45:00
3703 2024-05-19 01:45:00
3704 2024-05-19 02:45:00

[3705 rows x 1 columns]


In [88]:
# 생성한 날짜에 에어로졸 이미지 df 머지하기
merged_df = datetime_df.merge(img_df, left_on="DateTime", right_index=True, how="left")

In [89]:
merged_df.shape

(3705, 2)

In [91]:
# 에어로졸 이미지 결측인 부분
merged_df[merged_df.isna().any(axis=1)]

,DateTime,File
387,2021-05-10 00:45:00,NaN
388,2021-05-10 01:45:00,NaN
389,2021-05-10 02:45:00,NaN
1722,2022-07-29 00:45:00,NaN
1723,2022-07-29 01:45:00,NaN
1724,2022-07-29 02:45:00,NaN


In [95]:
pm_data.shape

(1235, 1)

In [98]:
# 미세먼지 데이터에서 이미지가 결측인 날짜 제거
pm_data = pm_data.drop("2021-05-10", axis = 0)
pm_data = pm_data.drop("2022-07-29", axis = 0)

In [107]:
# 기상 데이터에서 이미지가 결측인 날짜 제거
grouped_climate = grouped_climate.drop("2021-05-10", axis = 0)
grouped_climate = grouped_climate.drop("2022-07-29", axis = 0)

In [99]:
pm_data.shape

(1233, 1)

In [101]:
img_df.shape[0] / 3

1233.0

In [108]:
grouped_climate.shape

(1233, 2220)

In [110]:
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df.index.get_loc("2021-01-26 00:45:00"))
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df.index.get_loc("2021-10-08 02:45:00"))
# 보간적용한 사진 몇 번째 인덱스인지 확인하기
print(img_df.index.get_loc("2022-02-15 00:45:00"))

75
839
1227


In [103]:
# 인덱스 추출해두기
idx = pm_data.index

In [111]:
# 에어로졸 이미지 불러오기 및 전처리 함수
def preprocess_image(img_df, no_crop_dates_indice, base_path): # 이미지데이터프레임, 전처리안할 이미지 인덱스번호, 이미지 경로 설정
    img_vectors = {
        '0045': [], # 00시 45분 이미지 벡터를 담을 리스트
        '0145': [], # 01시 45분 이미지 벡터를 담을 리스트
        '0245': [] # 02시 45분 이미지 벡터를 담을 리스트
    }

    for i in range(len(img_df)):
        date = img_df.index[i]
        img_filename = img_df.iloc[i, 0][56:]  # 사용한 컴퓨터가 여러대여서 경로가 다 다르기때문에, 이미지파일명만 추출.
        img_path = os.path.join(base_path, img_filename) # 함수 호출 할때, 본인 컴퓨터내 에어로졸 이미지 경로는 base_path 자리에 설정.
        img = cv2.imread(img_path) # open cv 사용

        if img is not None:  # 이미지 로드가 성공했는지 확인
            if i not in no_crop_dates_indice:  # 인덱스가 no_crop_dates_indice에 있는지 확인
                img = img[150:1100, :]  # 위 아래 필요없는 정보 자르기
                img = img[:, 200:-200]  # 왼쪽 오른쪽 필요없는 정보 자르기
                img = img[400:1000, :]  # 추가 잘라내기 (필요시)

                # 이미지가 비어 있지 않은지 확인
                if img.size == 0: 
                    print(f"현재 {img_path} 이미지는 비어있습니다") # 만약 위의 크롭을 거치고 사이즈가 0이된 이미지가있는지 확인
                    continue # 0이 된게 있다면 스킵하기

            img = cv2.resize(img, (128, 128))  # 이미지 크기를 일관되게 조정
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR을 RGB로 변환
            img = img / 255.0  # 정규화 (0~1 범위로 변경하기)

            time_key = date.strftime('%H%M') # date (인덱스) 에서 시간과 분만 추출
            if time_key in img_vectors:
                img_vectors[time_key].append(img) # 각 이미지가 00:45인지 01:45인지 02:45인지 확인 후, 알맞는 리스트로 append

    return img_vectors


# 크기 조정 제외할 이미지들의 인덱스 (날짜) 이미 보간법 적용된 사진들.
no_crop_dates_indice = [72, 836, 1224] # 72번, 836번, 1224번 사진은 crop 전처리 거치지않기.

# 이미지 파일 경로
base_path = "C:/Users/user/Desktop/koreaIT/aerosol_image/new_images2/"  # 현재 컴퓨터내 이미지폴더 디렉토리

# 이미지 벡터화
img_vectors = preprocess_image(img_df, no_crop_dates_indice, base_path) # 함수 호출

# 데이터프레임 생성
result_df = pd.DataFrame({
    "index" : idx, # 추출해둔 인덱스로 date 생성
    '0045': img_vectors['0045'],
    '0145': img_vectors['0145'],
    '0245': img_vectors['0245']
})

In [112]:
result_df.shape

(1233, 4)

In [113]:
# 생성한 이미지 벡터 df의 인덱스를 datetime 형식으로 변경
result_df = result_df.set_index("index")
result_df.index = pd.to_datetime(result_df.index)

## 시퀀스 윈도우 함수 생성

In [114]:
# 기상데이터, 미세먼지데이터, 에어로졸 이미지데이터를 한번에 시퀀스 데이터로 변환해주는 함수
def create_combined_sliding_window(climate_data, pm_data, aod_df, past_days=3, future_days=3):
    x_climate, x_aod_0045, x_aod_0145, x_aod_0245, y = [], [], [], [], []
    
    for i in range(len(climate_data) - past_days - future_days + 1):
        # 과거 'past_days' 일의 기상 데이터
        climate_window = climate_data[i:i+past_days]
        # 미래 'future_days' 일의 미세먼지 데이터
        pm_window = pm_data[i+past_days:i+past_days+future_days]
        
        # 에어로졸 이미지의 슬라이딩 윈도우
        aod_window_0045 = aod_df['0045'].iloc[i:i + past_days]
        aod_window_0145 = aod_df['0145'].iloc[i:i + past_days]
        aod_window_0245 = aod_df['0245'].iloc[i:i + past_days]

        # 결측치가 없는 경우에만 추가
        if not (pd.DataFrame(climate_window).isna().any().any() or
                pd.DataFrame(pm_window).isna().any().any() or
                aod_window_0045.isnull().any() or
                aod_window_0145.isnull().any() or
                aod_window_0245.isnull().any()):
            
            x_climate.append(climate_window)
            x_aod_0045.append(np.stack(aod_window_0045.values))
            x_aod_0145.append(np.stack(aod_window_0145.values))
            x_aod_0245.append(np.stack(aod_window_0245.values))
            y.append(pm_window)

    return (np.array(x_climate),
            np.array(x_aod_0045),
            np.array(x_aod_0145),
            np.array(x_aod_0245),
            np.array(y))

In [115]:
# 데이터 준비
x_climate, x_0045, x_0145, x_0245, y_pm = create_combined_sliding_window(grouped_climate, pm_data, result_df, past_days=6, future_days=3)

In [116]:
# 데이터 분할 (훈련, 검증, 테스트셋)
x_climate_train, x_climate_test,  x_0045_train,  x_0045_test, x_0145_train, x_0145_test, x_0245_train, x_0245_test, y_train, y_test = train_test_split(x_climate, x_0045, x_0145, x_0245, y_pm, test_size=0.2, random_state=42)
x_climate_train, x_climate_val, x_0045_train,  x_0045_val, x_0145_train, x_0145_val, x_0245_train, x_0245_val, y_train, y_val = train_test_split(x_climate_train, x_0045_train, x_0145_train, x_0245_train, y_train, test_size=0.2, random_state=42)

print(f'train shapes: {x_climate_train.shape}, {x_0045_train.shape},{x_0145_train.shape},{x_0245_train.shape}, {y_train.shape}')
print(f'validation shapes: {x_climate_val.shape}, {x_0045_val.shape},{x_0145_val.shape},{x_0245_val.shape}, {y_val.shape}')
print(f'test shapes: {x_climate_test.shape}, {x_0045_test.shape},{x_0145_test.shape},{x_0245_test.shape}, {y_test.shape}')

train shapes: (772, 6, 2220), (772, 6, 128, 128, 3),(772, 6, 128, 128, 3),(772, 6, 128, 128, 3), (772, 3, 1)
validation shapes: (194, 6, 2220), (194, 6, 128, 128, 3),(194, 6, 128, 128, 3),(194, 6, 128, 128, 3), (194, 3, 1)
test shapes: (242, 6, 2220), (242, 6, 128, 128, 3),(242, 6, 128, 128, 3),(242, 6, 128, 128, 3), (242, 3, 1)


In [117]:
# 기상 데이터 스케일링
ss = StandardScaler()

In [118]:
# 3차원을 스케일링할 수 없기 때문에, 2차원으로 변환 후 다시 3차원으로 재변환
climate_train_reshaped = x_climate_train.reshape(x_climate_train.shape[0], -1)
climate_val_reshaped = x_climate_val.reshape(x_climate_val.shape[0], -1)
climate_test_reshaped = x_climate_test.reshape(x_climate_test.shape[0], -1)

climate_train_scaled_reshaped = ss.fit_transform(climate_train_reshaped)
climate_val_scaled_reshaped = ss.transform(climate_val_reshaped)
climate_test_scaled_reshaped = ss.transform(climate_test_reshaped)

scaled_climate_train = climate_train_scaled_reshaped.reshape(x_climate_train.shape)
scaled_climate_val = climate_val_scaled_reshaped.reshape(x_climate_val.shape)
scaled_climate_test = climate_test_scaled_reshaped.reshape(x_climate_test.shape)

In [119]:
# 3D CNN 정의
input_aerosol_0045 = Input(shape=(6, 128, 128, 3))
input_aerosol_0145 = Input(shape=(6, 128, 128, 3))
input_aerosol_0245 = Input(shape=(6, 128, 128, 3))

def create_cnn(input_layer):
    x = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling3D((1, 2, 2))(x) # 2, 2, 2 넣었더니 에러떠서 공간적 특징을 줄여서 1, 2 ,2로 설정.
    x = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((1, 2, 2))(x)
    x = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = MaxPooling3D((1, 2, 2))(x)
    x = Flatten()(x)
    return x

cnn_0045 = create_cnn(input_aerosol_0045)
cnn_0145 = create_cnn(input_aerosol_0145)
cnn_0245 = create_cnn(input_aerosol_0245)

# 기상 데이터 입력
input_weather = Input(shape=(6, scaled_climate_train.shape[2])) # 3, 2200이 들어감
weather_lstm = LSTM(128, activation='relu', return_sequences=False)(input_weather)

# 결합
combined = concatenate([cnn_0045, cnn_0145, cnn_0245, weather_lstm])
x = Dense(256, activation='relu')(combined)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='linear')(x)  # 미래 3일 예측

In [120]:
# 모델 정의
model = Model(inputs=[input_aerosol_0045, input_aerosol_0145, input_aerosol_0245, input_weather], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("./best_3D_CNN_model_pm25_seoul.h5", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)

In [121]:
# 모델확인
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 6, 128, 128, 3)]     0         []                            
                                                                                                  
 conv3d (Conv3D)             (None, 6, 128, 128, 32)      2624      ['input_1[0][0]']             
                                                                                              

In [122]:
# 모델 훈련
history = model.fit([x_0045_train, x_0145_train, x_0245_train, scaled_climate_train], y_train, epochs=20, batch_size=32,
                    validation_data=([x_0045_val, x_0145_val, x_0245_val, scaled_climate_val], y_val), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
25/25 [==============================] - ETA: 0s - loss: 786.4438

C:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 250s 10s/step - loss: 786.4438 - val_loss: 227.1125
Epoch 2/20
25/25 [==============================] - 246s 10s/step - loss: 137.0092 - val_loss: 178.2099
Epoch 3/20
25/25 [==============================] - 246s 10s/step - loss: 98.5449 - val_loss: 154.4013
Epoch 4/20
25/25 [==============================] - 246s 10s/step - loss: 69.9230 - val_loss: 119.3560
Epoch 5/20
25/25 [==============================] - 245s 10s/step - loss: 54.4338 - val_loss: 111.1721
Epoch 6/20
25/25 [==============================] - 244s 10s/step - loss: 37.9744 - val_loss: 95.5284
Epoch 7/20
25/25 [==============================] - 244s 10s/step - loss: 27.1015 - val_loss: 98.7946
Epoch 8/20
25/25 [==============================] - 245s 10s/step - loss: 20.4134 - val_loss: 86.5007
Epoch 9/20
25/25 [==============================] - 244s 10s/step - loss: 14.8005 - val_loss: 96.2396
Epoch 10/20
25/25 [==============================] - 245s 10s/step - loss: 12.1968 - v

In [124]:
# 테스트셋 MSE 추출
test_loss = model.evaluate([x_0045_test, x_0145_test, x_0245_test, scaled_climate_test], y_test)
test_loss

8/8 [==============================] - 6s 734ms/step - loss: 62.3636


62.3636360168457

In [128]:
# 예측 
y_pred = model.predict([x_0045_test, x_0145_test, x_0245_test, scaled_climate_test])

# 예측 결과 확인
print(f'예측값: \n {y_pred[:3]}')
print(f'실제값: \n {y_test[:3]}')

8/8 [==============================] - 6s 771ms/step
예측값: 
 [[16.588125 24.481993 34.441895]
 [16.734673 16.685986 27.528059]
 [11.390153 22.936243 25.380701]]
실제값: 
 [[[12.3]
  [24.7]
  [38.2]]

 [[12.4]
  [15.1]
  [21. ]]

 [[ 8.6]
  [11.5]
  [30.2]]]


## CNN 제외하고 LSTM으로만 돌려보기 (이미지 제외)

In [129]:
# 이미지를 제외했기때문에, 기상, 미세먼지 슬라이딩 윈도우 생성 함수 사용
# 기상, 미세먼지 슬라이딩 윈도우 생성 함수
def create_sliding_window(climate_data, pm_data, past_days=6, future_days=3):
    x_climate, y = [], []
    for i in range(len(climate_data) - past_days - future_days + 1):
        # 과거 'past_days' 일의 기상 데이터
        climate_window = climate_data[i:i+past_days]
        # 미래 'future_days' 일의 미세먼지 데이터
        pm_window = pm_data[i+past_days:i+past_days+future_days]

        # 결측치가 없는 경우에만 추가
        if not (pd.DataFrame(climate_window).isna().any().any() or pd.DataFrame(pm_window).isna().any().any()):
            x_climate.append(climate_window)
            y.append(pm_window)

    return np.array(x_climate), np.array(y)

In [130]:
# 함수 사용하여 시퀀스 데이터 생성
xx_climate, yy = create_sliding_window(grouped_climate, pm_data)

In [131]:
print(xx_climate.shape)
print(yy.shape)

(1208, 6, 2220)
(1208, 3, 1)


In [132]:
# 데이터 분할 (훈련, 검증, 테스트셋)
xx_climate_train, xx_climate_test, yy_train, yy_test = train_test_split(xx_climate, yy, test_size=0.2, random_state=42)
xx_climate_train, xx_climate_val, yy_train, yy_val = train_test_split(xx_climate_train, yy_train, test_size=0.2, random_state=42)

print(f'train shapes: {xx_climate_train.shape}, {yy_train.shape}')
print(f'validation shapes: {xx_climate_val.shape}, {yy_val.shape}')
print(f'test shapes: {xx_climate_test.shape}, {yy_test.shape}')

train shapes: (772, 6, 2220), (772, 3, 1)
validation shapes: (194, 6, 2220), (194, 3, 1)
test shapes: (242, 6, 2220), (242, 3, 1)


In [133]:
ss = StandardScaler()

In [134]:
# 기상데이터 스케일링
xx_climate_train_reshaped = xx_climate_train.reshape(xx_climate_train.shape[0], -1)
xx_climate_val_reshaped = xx_climate_val.reshape(xx_climate_val.shape[0], -1)
xx_climate_test_reshaped = xx_climate_test.reshape(xx_climate_test.shape[0], -1)

xx_climate_train_scaled_reshaped = ss.fit_transform(xx_climate_train_reshaped)
xx_climate_val_scaled_reshaped = ss.transform(xx_climate_val_reshaped)
xx_climate_test_scaled_reshaped = ss.transform(xx_climate_test_reshaped)

xx_scaled_climate_train = xx_climate_train_scaled_reshaped.reshape(xx_climate_train.shape)
xx_scaled_climate_val = xx_climate_val_scaled_reshaped.reshape(xx_climate_val.shape)
xx_scaled_climate_test = xx_climate_test_scaled_reshaped.reshape(xx_climate_test.shape)

In [137]:
# 기상 데이터 입력
input_weather = Input(shape=(6, 2220))

# 첫 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=True)(input_weather)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 두 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=True)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# 세 번째 LSTM 층
x = LSTM(128, activation='relu', return_sequences=False)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# Fully Connected (FC) 층
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(3, activation='linear')(x)  # 미래 3일 예측

# 모델 생성 및 컴파일
lstm_model = Model(inputs=input_weather, outputs=output)
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 요약
lstm_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 6, 2220)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 6, 128)            1202688   
                                                                 
 batch_normalization_1 (Bat  (None, 6, 128)            512       
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 6, 128)            0         
                                                                 
 lstm_3 (LSTM)               (None, 6, 128)            131584    
                                                                 
 batch_normalization_2 (Bat  (None, 6, 128)            512       
 chNormalization)                                          

In [138]:
# 콜백 설정 (모델 체크포인트, 조기 종료 등)
checkpoint_cb = ModelCheckpoint("./best_LSTM_model_pm25.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=8, restore_best_weights=True)

In [139]:
# 모델 훈련
history = lstm_model.fit(xx_scaled_climate_train, yy_train, epochs=100, batch_size=32,
                    validation_data=(xx_scaled_climate_val, yy_val), 
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
25/25 [==============================] - 3s 39ms/step - loss: 333.4209 - val_loss: 318.2815
Epoch 2/100
 5/25 [=====>........................] - ETA: 0s - loss: 137.6793

C:\Users\user\anaconda3\envs\koreait\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/25 [==============================] - 1s 33ms/step - loss: 131.0260 - val_loss: 270.0684
Epoch 3/100
25/25 [==============================] - 1s 32ms/step - loss: 122.9194 - val_loss: 254.9355
Epoch 4/100
25/25 [==============================] - 1s 32ms/step - loss: 119.9169 - val_loss: 227.6671
Epoch 5/100
25/25 [==============================] - 1s 33ms/step - loss: 108.6551 - val_loss: 194.3611
Epoch 6/100
25/25 [==============================] - 1s 32ms/step - loss: 98.1357 - val_loss: 183.8728
Epoch 7/100
25/25 [==============================] - 1s 33ms/step - loss: 100.0663 - val_loss: 162.9826
Epoch 8/100
25/25 [==============================] - 1s 30ms/step - loss: 91.2032 - val_loss: 164.0196
Epoch 9/100
25/25 [==============================] - 1s 30ms/step - loss: 96.3506 - val_loss: 215.9353
Epoch 10/100
25/25 [==============================] - 1s 30ms/step - loss: 81.7825 - val_loss: 301.1610
Epoch 11/100
25/25 [==============================] - 1s 31ms/step - loss: 82.2

In [140]:
test_loss = lstm_model.evaluate([xx_scaled_climate_test], yy_test)
test_loss

8/8 [==============================] - 0s 11ms/step - loss: 146.6371


146.63706970214844